### Set up LLM

In [1]:
#Install prerequisite packages
!pip install llama-index -q
!pip install llama-index-llms-groq -q
!pip install llama-index-embeddings-huggingface -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.6/263.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [2]:
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
import os
import nest_asyncio

#Used by LlamaIndex
nest_asyncio.apply()

#Used by LlamaIndex
nest_asyncio.apply()

from google.colab import userdata

#Setup the LLM
llm=Groq(model="llama-3.3-70b-versatile", api_key=userdata.get('GROQ_API_KEY'))

#Setup the embedding model RAG
Settings.embed_model= HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### 03.02. Create introspection agent

In [3]:
!pip install llama-index-agent-introspective -q

In [4]:
from llama_index.agent.introspective import SelfReflectionAgentWorker
from llama_index.agent.introspective import IntrospectiveAgentWorker
from llama_index.core.llms import ChatMessage, MessageRole

#Setup the reflection agent worker
self_reflection_agent_worker = SelfReflectionAgentWorker.from_defaults(
    llm=llm,
    verbose=True,
)

#Setup the introspective agent worker
introspective_agent_worker = IntrospectiveAgentWorker.from_defaults(
    reflective_agent_worker = self_reflection_agent_worker,
    main_agent_worker = None,
    verbose=True,
)

#Create a system prompt defining the function of the agent
system_prompt="""
You are an Product specification summarizer who can summarize a product specification.
For the input provided, create a summary with less than 50 words.

Ensurethat the summary focuses on performance specifications
and safety features.
"""
chat_history = [
    ChatMessage(
        content=system_prompt,
        role=MessageRole.SYSTEM
    )
]

#Create the agent
introspective_agent= introspective_agent_worker.as_agent(
    chat_history=chat_history,
    verbose=True
)

### 03.03. Execute the introspection Agent

In [5]:
!pip install pymupdf -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 88.7 MB/s eta 0:00:00


In [7]:
from llama_index.readers.file import PyMuPDFReader

#Input file for summarization
loader=PyMuPDFReader()
docs=loader.load(file_path="EcoSprint_Specification_Document.pdf")


#Pick the first page of the doc as content
source_content=docs[0].text
print(source_content)




EcoSprint Specification Document
1. Overview
●
The EcoSprint is a revolutionary electric vehicle (EV) designed for efficiency and
performance. With its sleek design and state-of-the-art technology, the EcoSprint
appeals to environmentally conscious drivers who don't want to compromise on style or
driving experience. Ideal for city driving and daily commutes, the EcoSprint offers a
perfect blend of comfort, sustainability, and innovation.
2. Design Specifications
●
Exterior Design: The EcoSprint boasts a modern and aerodynamic silhouette, featuring
smooth lines and a compact form factor. Available in colors like Midnight Black, Ocean
Blue, and Pearl White, it's a head-turner on the road.
●
Interior Design: Inside, the EcoSprint is a realm of comfort and luxury. It offers a
spacious cabin with seating for five, premium upholstery, and customizable ambient
lighting.
3. Performance Specifications
●
Engine and Motor: Powered by a high-efficiency electric motor, the EcoSprint delivers
200 ho

In [8]:
response=await introspective_agent.achat(source_content)

print("\n", response.response)

> Running step cda6a1eb-7dbc-497c-abea-72c7e806e64c. Step input: EcoSprint Specification Document
1. Overview
●
The EcoSprint is a revolutionary electric vehicle (EV) designed for efficiency and
performance. With its sleek design and state-of-the-art technology, the EcoSprint
appeals to environmentally conscious drivers who don't want to compromise on style or
driving experience. Ideal for city driving and daily commutes, the EcoSprint offers a
perfect blend of comfort, sustainability, and innovation.
2. Design Specifications
●
Exterior Design: The EcoSprint boasts a modern and aerodynamic silhouette, featuring
smooth lines and a compact form factor. Available in colors like Midnight Black, Ocean
Blue, and Pearl White, it's a head-turner on the road.
●
Interior Design: Inside, the EcoSprint is a realm of comfort and luxury. It offers a
spacious cabin with seating for five, premium upholstery, and customizable ambient
lighting.
3. Performance Specifications
●
Engine and Motor: Powered b